# Stock Sentiment Analysis Project

What's the goal of the project?
How do we plan to achieve the goal?
What are some technologies used?

# Setup

In [1]:
%matplotlib notebook

# import dependencies (for all of the notebook and remove from elsewhere)
import sys
import pandas as pd
from news_finders import content_scraper
from news_finders import newsApi
from yahoofinancials import YahooFinancials
from utils import helper_functions
from utils import sqlite_utils
from datetime import datetime, timedelta
from stock_historical_data import getStockData
from matplotlib import pyplot as plt 


connection_article_database = sqlite_utils.setUpArticleDatabase()

connection_pricing_database = sqlite_utils.setUpPricingDatabase()

max_article_date = helper_functions.getMaxDateForStockArticles()

## Gathering the Historical Price and Article Data for User Inputted Stock Tickers
Utilizing the News API service, we gather articles referencing the Stock Symbols that we are interested in. We t ake the JSON data returned, clean it, and store into our sqlite database for future reference.

In [3]:
#STOCKS = ["WMT", "AMZN", "MRNA"]
STOCKS = helper_functions.getStockTickersFromUser()


newsApi.getAndStoreArticlesForMultipleStocks(STOCKS, max_article_date, connection_article_database)


getStockData.getAndStoreMultipleStocksPricingData(STOCKS, connection_pricing_database)

Input individual stock ticker to analyze or multiple comma seperated stock tickers. If done, write 'done'
ZIM
Input individual stock ticker to analyze or multiple comma seperated stock tickers. If done, write 'done'
done
None
Failed to create article UNIQUE constraint failed: stockArticles.stockSymbol, stockArticles.name Article(stockSymbol='ZIM', name="The 'Value' Strikes Back - Value Strategies' Performance Through COVID", url='https://seekingalpha.com/article/4417747-value-strikes-back-value-strategies-performance-through-covid?source=feed', content='Photo by matejmo/iStock via Getty Images\r\nIncredibly, it’s been more than 1 year since COVID-19 started wreaking havoc on the world at large, and by extension the financial markets. While it has been… [+24242 chars]', description='We look at performance of the following strategies over the last year: Net-nets, Low EV/EBIT, the Magic Formula, and Crisis Value, among others.', scraper='NewsAPI', date='2021-04-06')
Failed to create artic

In [ ]:
# Test to show the stock article and stock pricing database contents
sqlite_utils.fetchStockDataOverview(connection_pricing_database, connection_article_database, STOCKS)

### Pulling Stock Article and Sentiment Data from stockSentiment Database

In [ ]:
dataFrame = pd.read_sql_query('SELECT * FROM stockArticles WHERE date >= "2020-03-16" AND date <= "2020-11-14" AND stockSymbol="AAPL" ORDER BY date DESC', connection_article_database)

dataFrame

### Pulling Stock Price Data from stockPricing Database and Plotting the Data

In [ ]:
data = sqlite_utils._findAllStockPricingForStockSymbol(connection_pricing_database, "WMT")

# Adding Title 
plt.title("Stock Prices")
  
# Labeling the axes 
plt.xlabel("Date") 
plt.ylabel("Closing Price in $")
     
    
xs = [x[0] for x in data]
ys = [y[1] for y in data]

#print("xs: ", xs)
#print("ys: ", ys)
#plt.autoscale(enable=True, axis='both', tight=None)

if x
# Function to plot  
plt.plot(xs, ys)

# function to show the plot  
plt.show()

# Perform sentiment analysis on the articles